GoogleDriveをマウントして、自分のディレクトリとして扱う。

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


ディレクトリの中身を確認

In [0]:
!ls gdrive/My\ Drive/NLP

corpus	normal.hdf5


Chainerをインストールしましょう。

In [0]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   5808      0 --:--:-- --:--:-- --:--:--  5808
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!


ライブラリのインポート

In [0]:
import numpy as np
import codecs
import sys
import chainer
from chainer import training, datasets, iterators, optimizers
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

RNNモデル定義

In [0]:
class Generate_RNN(chainer.Chain):
	def __init__(self, words_size, nodes):
		super(Generate_RNN, self).__init__()
		with self.init_scope():
			# Embed -> 埋め込み
			# LSTM -> Long Sort Term Memory
			self.embed = L.EmbedID(words_size, words_size)
			self.l1 = L.LSTM(words_size, nodes)
			self.l2 = L.LSTM(nodes, nodes)
			self.l3 = L.Linear(nodes, words_size)

	def reset_state(self):
		self.l1.reset_state()
		self.l2.reset_state()

	def __call__(self, x):
		h0 = self.embed(x)
		h1 = self.l1(h0)
		h2 = self.l2(h1)
		y = self.l3(h2)
		return y

In [0]:
class RNNUpdater(training.StandardUpdater):
	def __init__(self, train_iter, optimizer, device, cp):
		super(RNNUpdater, self).__init__(
			train_iter,
			optimizer,
			device=device,
		)
		self.cp = cp

	def update_core(self):
		loss = 0

		# IteratorとOptimizerの取得
		train_iter = self.get_iterator("main")
		optimizer = self.get_optimizer("main")

		# Modelの取得
		model = optimizer.target
		# 文をバッチ取得(足りないものを埋める)
		x = train_iter.__next__()
		# モデルのステータスをリセット
		model.reset_state()

		# 文の単語をRNNに学習させる
		for i in range(len(x[0])-1):
			# バッチ処理の配列
			batch = self.cp.array([s[i] for s in x], dtype=self.cp.int32)
			# 正解ラベル
			t = self.cp.array([s[i+1] for s in x], dtype=self.cp.int32)

			#終端文字なら終わり
			if self.cp.min(batch) == 1 and self.cp.max(batch) == 1:
				break

			# RNNを１回実行
			y = model(batch)
			# 損失を求める
			loss += F.softmax_cross_entropy(y, t)

		# 逆伝播
		optimizer.target.cleargrads()
		loss.backward()
		optimizer.update()


GPUの設定

In [0]:
uses_device = 0

cp = np
if uses_device >= 0:
  import cupy as cp
  import chainer.cuda
  
if uses_device >= 0:
	chainer.cuda.get_device_from_id(0).use()
	chainer.cuda.check_cuda_available()
	model.to_gpu()

読み込み、保存ファイル名の定義

In [0]:
train_data = './gdrive/My Drive/NLP/corpus/all-sentence.txt'
conb_data = './gdrive/My Drive/NLP/corpus/all-words.txt'
model_file = './gdrive/My Drive/NLP/normal.hdf5'

単語リストの読み込み

In [0]:
w = codecs.open(conb_data, "r", "utf-8")
words = {}

In [0]:
# 行の中の単語をリスト化
line = w.readline()
while line:
	l = line.split(",")
	if len(l) >= 2:
		words[int(l[0])] = l[1].strip()
	line = w.readline()
w.close()

モデルの生成


In [0]:
model = Generate_RNN(len(words)+2, 200)

# 学習結果の読み込み
chainer.serializers.load_hdf5(model_file, model)

各種パラメータ

In [0]:
words_max = 50
beam_w = 10
sentence = []
model_history = [model]
cur_sentence = [0]
cur_score = []
max_score = 0

木探索の関数定義

In [0]:
def Tree_Traverse():
	global max_score

	cur_word = cur_sentence[-1]
	score = np.prod(cur_score)
	deep = len(cur_sentence)

	# 枝狩り
	if deep > 5 and max_score*0.6 > score:
		return

	# 終端文字または最大の文の長さ以上なら、文を追加して終了
	if cur_word == 1 or deep > words_max:
		data = np.array(cur_sentence)
		sentence.append((score, data))
		if max_score < score:
			max_score = score
		return

	cur_model = model_history[-1].copy()
	x = cp.array([cur_word], dtype=cp.int32)
	y = cur_model(x)
	z = F.softmax(y)
	result = z.data[0]

	if uses_device >= 0: result = chainer.cuda.to_cpu(result)
	p = np.argsort(result)[::-1]

	model_history.append(cur_model)
	for i in range(beam_w):
		cur_sentence.append(p[i])
		cur_score.append(result[p[i]])
		Tree_Traverse()
		cur_sentence.pop()
		cur_score.pop()
	model_history.pop()

木探索の実行および、スコアの高い文章の表示

In [0]:
# 木探索で探索で文章を作成
Tree_Traverse()

# スコアの高いものから順に表示
result_set = sorted(sentence, key=lambda x: x[0])[::-1]

for i in range(min([20, len(result_set)])):
	s, l = result_set[i]
	r = str(s) + "\t"
	for w in l:
		if w > 1:
			r += words[w]
	r += "\n"

	print(r)

0.00018694038	キャロル自筆の原本は現在大英博物館に収蔵されている

0.00018608986	アリスがそれを飲むと、身長が約に縮んだ

3.3501434e-05	アリスはどうやら夢を見ていた

2.7118393e-05	アリスがそれを食べると、今度は身体が大きくなりすぎてしまう

2.1547747e-05	アリスは、陪審員の動物たちに混じって裁判を見物する

1.5762826e-05	アリスはどうやら夢を見ていたらしい

1.22637775e-05	『パブリッシャー・サーキュラー』

9.561816e-06	またパロディ詩

6.175095e-06	また児童小説

3.0908795e-06	前作同様

2.7796455e-06	また児童

1.936947e-06	また児童文学

1.4593134e-06	この話に対して

1.4455303e-06	前作から同様

1.0002511e-06	年の月

5.983845e-07	また児童パロディ

4.3602267e-07	この話も

3.3054778e-07	アリスに言わ

2.9289805e-07	またパロディ本

2.4298518e-07	『パブリッシャー・サーキュラー）

